In [1]:
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

dill 0.3.8 is installed but dill<0.3.2,>=0.3.1.1 is required
Redoing requirement with just package name...
cloudpickle 3.1.0 is installed but cloudpickle~=2.2.1 is required
Redoing requirement with just package name...
numpy 1.26.4 is installed but numpy<1.25.0,>=1.14.3 is required
Redoing requirement with just package name...
pyarrow 17.0.0 is installed but pyarrow<10.0.0,>=3.0.0 is required
Redoing requirement with just package name...
jupyterlab 4.3.1 is installed but jupyterlab~=3.6.0 is required
Redoing requirement with just package name...
google-cloud-bigquery 2.34.4 is installed but google-cloud-bigquery[bqstorage,pandas]>=3.10.0 is required
Redoing requirement with just package name...
google-cloud-storage 1.44.0 is installed but google-cloud-storage>=2.0.0 is required
Redoing requirement with just package name...
pandas 2.2.3 is installed but pandas<2.1.4,>=1.5.0 is required
Redoing requirement with just package name...
numpy 1.26.4 is installed but numpy<3.0.0,>=2.0.0 is req

In [ ]:
import wandb
from huggingface_hub import login

hf_token = ""
wanb_token = ""

# Login to Hugging Face and Weights & Biases
login(token=hf_token)
wandb.login(key=wanb_token)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch
from unsloth import FastLanguageModel

max_seq_length = 5096 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [5]:
from datasets import load_dataset
dataset = load_dataset("lighteval/MATH", split="train", trust_remote_code = True)

MATH.py:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

data/algebra_train.jsonl:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)ata/counting_and_probability_train.jsonl:   0%|          | 0.00/707k [00:00<?, ?B/s]

data/geometry_train.jsonl:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

data/intermediate_algebra_train.jsonl:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

data/number_theory_test.jsonl:   0%|          | 0.00/376k [00:00<?, ?B/s]

data/prealgebra_test.jsonl:   0%|          | 0.00/553k [00:00<?, ?B/s]

data/precalculus_test.jsonl:   0%|          | 0.00/614k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
def formatting_prompts_func(example):
    messages = [
    {"role": "user", "content": f"Problem: {example['problem']}\n\nSolve this math problem step by step."},
    {"role": "assistant", "content": example['solution']},
    ]
    texts = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
    return {"task": texts}

dataset = dataset.map(formatting_prompts_func, batched=False)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [7]:
def tokenize_function(example):
    outputs = tokenizer(example["task"], truncation=True, padding="max_length", max_length=5096)
    return outputs
    
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["problem", "solution", "level", "type"], load_from_cache_file=False)
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets

Dataset({
    features: ['task', 'input_ids', 'attention_mask'],
    num_rows: 7500
})

In [9]:
trained_model_name = "Llama-1B-Math-LoRA-r32"
huggingface_username = "bharati2324"

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_datasets,
    dataset_text_field = "task",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = False,
        logging_steps = 20,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        save_strategy="steps",  # Choose "steps" or "epoch"
        save_steps=200,
        output_dir = "outputs",
        report_to = "wandb",  # Reporting to Weights & Biases
        push_to_hub=True,  # Push the model to Hugging Face Hub
        hub_model_id= f"{huggingface_username}/{trained_model_name}",  # Set this to your Hugging Face model name
        hub_strategy="all_checkpoints"
    ),
)

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 937
 "-____-"     Number of trainable parameters = 90,177,536
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: bpanigra (bpanigra-usc). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241215_034427-ec198dru
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run outputs
wandb: ⭐️ View project at https://wandb.ai/bpanigra-usc/huggingface
wandb: 🚀 View run at https://wandb.ai/bpanigra-usc/huggingface/runs/ec198dru


Step,Training Loss
20,1.291300
40,0.896200
60,0.861500
80,0.763200
100,0.781200
120,0.766200
140,0.675400
160,0.759900
180,0.738100
200,0.795000


In [13]:
model.save_pretrained(f"/kaggle/working/{trained_model_name}") 
tokenizer.save_pretrained(f"/kaggle/working/{trained_model_name}")

('/kaggle/working/Llama-1B-Math-LoRA-r32/tokenizer_config.json',
 '/kaggle/working/Llama-1B-Math-LoRA-r32/special_tokens_map.json',
 '/kaggle/working/Llama-1B-Math-LoRA-r32/tokenizer.json')

In [14]:
model.push_to_hub(trained_model_name) # Online saving
tokenizer.push_to_hub(trained_model_name) # Online saving

Saved model to https://huggingface.co/Llama-1B-Math-LoRA-r32


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [15]:
merged_model_name = f"{huggingface_username}/{trained_model_name}-merged"
model.push_to_hub_merged(merged_model_name, tokenizer)

Unsloth: You are pushing to hub in Kaggle environment.
To save memory, we shall move bharati2324/Llama-1B-Math-LoRA-r32-merged to /tmp/Llama-1B-Math-LoRA-r32-merged
Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 19.21 out of 31.35 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 40.18it/s]


Unsloth: Saving tokenizer...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

 Done.
Unsloth: Saving /tmp/Llama-1B-Math-LoRA-r32-merged/pytorch_model.bin...


pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/bharati2324/Llama-1B-Math-LoRA-r32-merged


In [16]:
#Inference
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

new_model_name = merged_model_name
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", 
     "content": "Problem: Find the value of x if 2x + 5 = 13.\n\nSolve this math problem step by step."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True,
                         temperature = 0.7, min_p = 0.1)
print(tokenizer.batch_decode(outputs)[0])

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Dec 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Problem: Find the value of x if 2x + 5 = 13.

Solve this math problem step by step.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

We have $2x + 5 = 13$.  Subtracting 5 from both sides, we have $2x = 8$.  Dividing both sides by 2, we have $x = 4$.  Therefore, the value of $x$ is $\boxed{4}$.<|eot_id|>
